Collaborative filtering

In [2]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
import pandas as pd

# 读取 ratings_small.csv
ratings = pd.read_csv('ratings_small.csv')

# 使用 Surprise 格式化数据
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# 使用 SVD 算法（奇异值分解）
model = SVD()

# 交叉验证模型性能
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/ee541/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/ee541/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/ee541/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/o

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [ ]:
from surprise import train_test_split

# 划分训练集和测试集
trainset, testset = train_test_split(data, test_size=0.2)

# 训练模型
model.fit(trainset)

# 给某个用户推荐没有看过的电影
def get_top_n_recommendations(predictions, n=5):
    from collections import defaultdict
    top_n = defaultdict(list)
    
    # 将所有预测值按 user 分类
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    
    # 每个 user 取预测评分最高的前 n 部电影
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    
    return top_n

# 预测测试集评分
predictions = model.test(testset)
top_n = get_top_n_recommendations(predictions, n=5)

# 示例：打印 userId=1 的推荐
print(f"Top 5 movies recommended for user 1: {top_n[1]}")
